In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 12, 6
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingRegressor

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
s = pd.read_csv('tmp/grouped.csv').sample(100000)

In [3]:
tr = s[s.Expected != -1]
te = s[s.Expected == -1]

In [4]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62100 entries, 556207 to 963621
Data columns (total 92 columns):
Id                            62100 non-null float64
Expected                      62100 non-null float64
cnt                           62100 non-null float64
null_ratio                    62100 non-null float64
minutes_past_mean             62100 non-null float64
minutes_past_min              62100 non-null float64
minutes_past_max              62100 non-null float64
minutes_past_std              62091 non-null float64
radardist_km_mean             62100 non-null float64
radardist_km_min              62100 non-null float64
radardist_km_max              62100 non-null float64
radardist_km_std              62091 non-null float64
Ref_mean                      38418 non-null float64
Ref_min                       38418 non-null float64
Ref_max                       38418 non-null float64
Ref_std                       36017 non-null float64
Ref_5x5_10th_mean             33308 n

In [5]:
base_metrics = [
    'Ref',
    'RefComposite',
    'RhoHV',
    'Zdr',
    'Kdp'
]
metric_modifiers = [
    '',
    '_5x5_10th',
    '_5x5_50th',
    '_5x5_90th'
]
metric_stats = [
    '_mean',
    '_min',
    '_max'
]
for bm1 in base_metrics:
    for mm1 in metric_modifiers:
        for ms1 in metric_stats:
            for bm2 in base_metrics:
                for mm2 in metric_modifiers:
                    for ms2 in metric_stats:
                        m1 = bm1 + mm1 + ms1
                        m2 = bm2 + mm2 + ms2
                        tr[m1 + '_per_' + m2] = tr[m1] / tr[m2]

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62100 entries, 556207 to 963621
Columns: 3692 entries, Id to Kdp_5x5_90th_max_per_Kdp_5x5_90th_max
dtypes: float64(3692)
memory usage: 1.7 GB


In [7]:
trl = tr.Expected
del tr['Expected']

In [8]:
tr = tr.replace([np.inf, -np.inf], 0)
tr = tr.fillna(0)

In [9]:
regr = GradientBoostingRegressor(n_estimators=300, max_depth=5, )

In [10]:
regr.fit(tr, trl)

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=5, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=300,
             random_state=None, subsample=1.0, verbose=0, warm_start=False)

In [11]:
importances = regr.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(100):
    print("%d. feature %d (%f) %s" % (f + 1, indices[f], importances[indices[f]], tr.columns[indices[f]]))

1. feature 0 (0.153560) Id
2. feature 3 (0.051359) minutes_past_mean
3. feature 6 (0.044215) minutes_past_std
4. feature 2 (0.033740) null_ratio
5. feature 9 (0.032285) radardist_km_max
6. feature 8 (0.026766) radardist_km_min
7. feature 7 (0.026355) radardist_km_mean
8. feature 1 (0.021750) cnt
9. feature 4 (0.020349) minutes_past_min
10. feature 5 (0.020017) minutes_past_max
11. feature 108 (0.010430) Ref_mean_per_RefComposite_5x5_10th_max
12. feature 28 (0.006948) RefComposite_min
13. feature 74 (0.006834) Zdr_5x5_90th_std
14. feature 1362 (0.006759) RefComposite_5x5_90th_mean_per_Ref_5x5_90th_max
15. feature 2946 (0.005735) Zdr_5x5_90th_max_per_RhoHV_5x5_90th_max
16. feature 954 (0.005535) RefComposite_max_per_RefComposite_5x5_90th_max
17. feature 2886 (0.005377) Zdr_5x5_90th_min_per_RhoHV_5x5_90th_max
18. feature 2826 (0.005127) Zdr_5x5_90th_mean_per_RhoHV_5x5_90th_max
19. feature 25 (0.004989) Ref_5x5_90th_max
20. feature 2178 (0.004347) RhoHV_5x5_90th_min_per_Zdr_5x5_90th_max
21